# 3 Data Preprocessing (Price)

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pickle

In [3]:
# read dataset
sales_test_val = pd.read_pickle('dataset/sales_test_val.pkl')
calendar = pd.read_pickle('dataset/calendar.pkl')
sell_prices = pd.read_pickle('dataset/sell_price.pkl')
matrix = pd.read_pickle('dataset/simple_matrix.pkl')

In [6]:
df_price = sales_test_val
df_price = df_price.set_index("id")
df_price = df_price.iloc[:, 5:].stack().reset_index()


In [7]:
df_price

,id,level_1,0
0,HOBBIES_1_001_CA_1_evaluation,d_1,0
1,HOBBIES_1_001_CA_1_evaluation,d_2,0
2,HOBBIES_1_001_CA_1_evaluation,d_3,0
3,HOBBIES_1_001_CA_1_evaluation,d_4,0
4,HOBBIES_1_001_CA_1_evaluation,d_5,0
...,...,...,...
59181085,FOODS_3_827_WI_3_evaluation,d_1937,0
59181086,FOODS_3_827_WI_3_evaluation,d_1938,2
59181087,FOODS_3_827_WI_3_evaluation,d_1939,2
59181088,FOODS_3_827_WI_3_evaluation,d_1940,5


In [9]:
df_price = pd.merge(df_price, calendar.iloc[:, 1:7], left_on='level_1', right_on='d')
sell_prices['store_id'] = sell_prices['item_id'].str.cat(sell_prices['store_id'], sep="_") + "_evaluation"
sell_prices = sell_prices.drop("item_id", axis=1)
df_price = pd.merge(df_price, sell_prices, left_on=['wm_yr_wk', "id"], right_on=['wm_yr_wk', "store_id"])
df_price = df_price.iloc[:, [8, 9, 10]]
df_price

In [ ]:
df_price.to_pickle('dataset/df_price_train.pkl')

In [ ]:
list_index = pd.unique(calendar["d"])
df = pd.DataFrame({ 'd' : list_index})
list_index = pd.unique(df_price["store_id"])

for i in range(len(pd.unique(list_index))):
    temp = df_price[df_price["store_id"] == list_index[i]]
    temp = temp.pivot_table(values=['sell_price'], index=['d'], columns=['store_id'], aggfunc='sum')
    temp.columns=temp.columns.get_level_values(1)
    temp = temp.reset_index()
    df =  pd.merge(df, temp, how = "left")
    
df = df.set_index("d")

In [ ]:
list_index = pd.unique(calendar["d"].tail(28))
df_predict = pd.DataFrame({ 'd' : list_index})
df_predict = pd.merge(df_predict, calendar.iloc[:, 1:7])
item_index = sales_test_val.columns.values

In [ ]:
for i in range(len(df.columns.values)):
    r = sell_prices[sell_prices["store_id"] == df.columns.values[i]]
    r = r.rename(columns={'sell_price': df.columns.values[i]})
    df_predict = pd.merge(df_predict, r.iloc[:, 1:3])
df_predict = df_predict.set_index("d")
df_predict =df_predict.iloc[:, 5:]

In [ ]:
df = pd.concat([df, df_predict], axis=0)

In [ ]:
# 出力
df.to_pickle('dataset/df_price_predict.pkl')